## Summary of this notebook

**Feature Engineering:**  
Following features/predictors are doesn't seem to influence churn, hence will be dropped.  
* State : There are more than 20 states and churn in each state is almost equal to average churn rate, hence not useful.  
* Phone : The phone number is a unique number and will not affect the churn rate  
* Area code : This feature also will not affect churn  
* Day Charge is a function of Day Calls and Day Mins, hence dropping 'Day Charge'   
* Also dropping 'Eve Charge', 'Night Charge' and 'International Charge' for similar reason   


#### Establishing baseline scores using Naive Baye's Classifiers
Base line score is around 85% - Using Naive Baye's ML

#### Using Random Forest ML - Using cross validation, without hyper parameter tuning
RF accuracy is around 95%, 10 percentage points better than Naive Baye's

#### Using XGBoost, with learning_rate = 0.1, no_of_estimators=200, focusing on 'recall'
With XGBoost, with given parameters, the accuracy improved by 1%, to 96%

#### Hyper Parameter tuning, on RF based model. Using RandomSearchCV for best parameters
Hyper Parameter tuned RF resulted in 96.1% accuracy, marginal improvement over XGBoost

**Further scope of analysis:**   
These classification models assume threshold of 0.5 to classify the outcome.  
It is ok to mis classify a satisfied customer as unsatisfied customer, but not vice versa.  
This is called recall in ML jargon.  
To achieve good recall, we need to re assess our model by chaning classification threshold to values other than 0.5.  
It is also noted that as recall improves, the accuracy of the model decreases.  
We need to comeup with right threshold for which accuracy and recall are tolerable.  
The ML trained object, say model.predict() returns the prediction for the given data with 0.5 as threshold.  
We can use model.predict_proba() to know the probability of prediction being class_1 or class_0.  

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('TelecomChurnColumnsRenamed.xls', index_col='Unnamed: 0')

In [3]:
df.shape

(4617, 21)

In [4]:
df.head(2)

,State,Account Length,Area Code,Phone,International Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,International Mins,International Calls,International Charge,CustServ Calls,Churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.7,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.7,1,False.


In [5]:
cat_columns = df.select_dtypes(include=['object']).columns.tolist()

for col in cat_columns:
    df[col] = df[col].str.strip()
    df[col] = df[col].astype('category') 

#Changin 'Area Code' column as 'category' type
df['Area Code'] = df['Area Code'].astype('category') 

#Updated cat_columns
cat_columns = df.select_dtypes(include=['category']).columns.tolist()
print(cat_columns)

['State', 'Area Code', 'Phone', 'International Plan', 'VMail Plan', 'Churn']


#### Defining Target variable

In [6]:
df['CHURN-FLAG'] = df.Churn.apply(lambda x: int(1) if x == 'True.' else int(0))

In [7]:
print(df.shape)
df.head(2)

(4617, 22)


,State,Account Length,Area Code,Phone,International Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Charge,Night Mins,Night Calls,Night Charge,International Mins,International Calls,International Charge,CustServ Calls,Churn,CHURN-FLAG
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,16.78,244.7,91,11.01,10.0,3,2.7,1,False.,0
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,16.62,254.4,103,11.45,13.7,3,3.7,1,False.,0


#### Feature Engineering

In [8]:
#Phone number seems to be unrelated to churn, hence dropping
df1 = df.drop('Phone', axis=1)
df1.shape

(4617, 21)

In [9]:
df1.columns

Index(['State', 'Account Length', 'Area Code', 'International Plan',
       'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'International Mins', 'International Calls',
       'International Charge', 'CustServ Calls', 'Churn', 'CHURN-FLAG'],
      dtype='object')

In [10]:
#Day Charge is a function of Day Calls and Day Mins, hence dropping 'Day Charge', 
#also dropping 'Eve Charge', 'Night Charge' and 'International Charge' for similar reason.
df1.drop(['Day Charge', 'Eve Charge', 'Night Charge', 'International Charge', ], axis=1, inplace=True)
df1.shape

(4617, 17)

In [11]:
#'Churn' having values 'True.' and 'False.' already captured in 'CHURN-FLAG'
df1.drop('Churn', axis=1, inplace=True)
df1.shape

(4617, 16)

In [12]:
df1.select_dtypes(include=['category']).columns.tolist()

['State', 'Area Code', 'International Plan', 'VMail Plan', 'CHURN-FLAG']

In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4617 entries, 0 to 4616
Data columns (total 16 columns):
State                  4617 non-null category
Account Length         4617 non-null int64
Area Code              4617 non-null category
International Plan     4617 non-null category
VMail Plan             4617 non-null category
VMail Message          4617 non-null int64
Day Mins               4617 non-null float64
Day Calls              4617 non-null int64
Eve Mins               4617 non-null float64
Eve Calls              4617 non-null int64
Night Mins             4617 non-null float64
Night Calls            4617 non-null int64
International Mins     4617 non-null float64
International Calls    4617 non-null int64
CustServ Calls         4617 non-null int64
CHURN-FLAG             4617 non-null category
dtypes: category(5), float64(4), int64(7)
memory usage: 458.7 KB


In [14]:
df1.head()

,State,Account Length,Area Code,International Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,International Mins,International Calls,CustServ Calls,CHURN-FLAG
0,KS,128,415,no,yes,25,265.1,110,197.4,99,244.7,91,10.0,3,1,0
1,OH,107,415,no,yes,26,161.6,123,195.5,103,254.4,103,13.7,3,1,0
2,NJ,137,415,no,no,0,243.4,114,121.2,110,162.6,104,12.2,5,0,0
3,OH,84,408,yes,no,0,299.4,71,61.9,88,196.9,89,6.6,7,2,0
4,OK,75,415,yes,no,0,166.7,113,148.3,122,186.9,121,10.1,3,3,0


In [15]:
df1['CHURN-FLAG'] = df1['CHURN-FLAG'].astype(int)

In [16]:
from sklearn.preprocessing import LabelEncoder
df1.State = LabelEncoder().fit_transform(df1.State)
df1['Area Code'] = LabelEncoder().fit_transform(df1['Area Code'])
df1['International Plan'] = LabelEncoder().fit_transform(df1['International Plan'])
df1['VMail Plan'] = LabelEncoder().fit_transform(df1['VMail Plan'])

In [17]:
df1.head()

,State,Account Length,Area Code,International Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,International Mins,International Calls,CustServ Calls,CHURN-FLAG
0,16,128,1,0,1,25,265.1,110,197.4,99,244.7,91,10.0,3,1,0
1,35,107,1,0,1,26,161.6,123,195.5,103,254.4,103,13.7,3,1,0
2,31,137,1,0,0,0,243.4,114,121.2,110,162.6,104,12.2,5,0,0
3,35,84,0,1,0,0,299.4,71,61.9,88,196.9,89,6.6,7,2,0
4,36,75,1,1,0,0,166.7,113,148.3,122,186.9,121,10.1,3,3,0


In [18]:
X = df1.iloc[:,:-1]

In [19]:
y = df1['CHURN-FLAG']

## Establishing baseline scores using Naive Baye's Classifiers

In [25]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [26]:
model_gaussiannb = GaussianNB()

model_gaussiannb.fit(X_train, y_train)
y_predict = model_gaussiannb.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(y_test, y_predict)

0.8484848484848485

In [28]:
model_bernoullinb = BernoulliNB()

model_bernoullinb.fit(X_train, y_train)
y_predict = model_bernoullinb.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(y_test, y_predict)

0.8427128427128427

#### Base line score is around 85% - Using Naive Baye's ML

## Using Random Forest ML - Using cross validation, without hyper parameter tuning

In [20]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state=10)

In [21]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_rf,X,y,
                         cv=10,
                         #scoring='recall',
                         scoring='accuracy',
                        )
print(scores)
print(scores.mean())

[0.93722944 0.94588745 0.92857143 0.96320346 0.95238095 0.94588745
 0.94805195 0.95661605 0.94143167 0.96095445]
0.9480214290409519


#### RF accuracy is around 95%, 10 percentage points better than Naive Baye's

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [24]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3231, 15)
(1386, 15)
(3231,)
(1386,)


## Using XGBoost, with learning_rate = 0.1, no_of_estimators=200, focusing on 'recall'

In [30]:
from xgboost import XGBClassifier

In [31]:
model_xg = XGBClassifier(learning_rate=0.1, random_state=0, n_estimators=200, scoring='recall')
model_xg.fit(X_train, y_train)
y_predict = model_xg.predict(X_test)

In [32]:
accuracy_score(y_test, y_predict)

0.9595959595959596

In [33]:
pd.crosstab(y_test, y_predict)

col_0,0,1
CHURN-FLAG,,
0,1193,7
1,49,137


#### With XGBoost, with given parameters, the accuracy improved by 1%, to 96%

## Hyper Parameter tuning, on RF based model. Using RandomSearchCV for best parameters

In [34]:
#model
MOD = RandomForestClassifier() 
#Implemente RandomSearchCV
m_params = { 
    "n_estimators" : np.linspace(2, 500, 500, dtype = "int"),  
    "max_depth": [5, 20, 30, None], 
    "min_samples_split": np.linspace(2, 50, 50, dtype = "int"),  
    "max_features": ["sqrt", "log2",10, 20, None],
    "oob_score": [True],
    "bootstrap": [True]
}
scoreFunction = {"recall": "recall", "precision": "precision"}

from sklearn.model_selection import RandomizedSearchCV
    
random_search = RandomizedSearchCV(MOD,
                                       param_distributions = m_params, 
                                       n_iter = 20,
                                       scoring = scoreFunction,               
                                       refit = "recall",
                                       return_train_score = True,
                                       random_state = 42,
                                       cv = 5,
                                       verbose = 5) 

In [35]:
#trains and optimizes the model
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] oob_score=True, n_estimators=459, min_samples_split=44, max_features=None, max_depth=5, bootstrap=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  oob_score=True, n_estimators=459, min_samples_split=44, max_features=None, max_depth=5, bootstrap=True, precision=(train=0.942, test=0.855), recall=(train=0.739, test=0.691), total=   7.5s
[CV] oob_score=True, n_estimators=459, min_samples_split=44, max_features=None, max_depth=5, bootstrap=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.7s remaining:    0.0s


[CV]  oob_score=True, n_estimators=459, min_samples_split=44, max_features=None, max_depth=5, bootstrap=True, precision=(train=0.957, test=0.921), recall=(train=0.771, test=0.617), total=   7.6s
[CV] oob_score=True, n_estimators=459, min_samples_split=44, max_features=None, max_depth=5, bootstrap=True 


...

C:\Users\91984\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)


[CV]  oob_score=True, n_estimators=366, min_samples_split=9, max_features=20, max_depth=30, bootstrap=True, precision=(train=nan, test=nan), recall=(train=nan, test=nan), total=   0.4s
[CV] oob_score=True, n_estimators=366, min_samples_split=9, max_features=20, max_depth=30, bootstrap=True 


C:\Users\91984\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  8.0min finished


[CV]  oob_score=True, n_estimators=366, min_samples_split=9, max_features=20, max_depth=30, bootstrap=True, precision=(train=nan, test=nan), recall=(train=nan, test=nan), total=   0.5s


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [36]:
#recover the best model
MOD = random_search.best_estimator_

In [37]:
#MOD.fit(X_train, y_train)
y_predict = MOD.predict(X_test)

In [38]:
accuracy_score(y_test, y_predict)

0.961038961038961

#### Hyper Parameter tuned RF resulted in 96.1% accuracy, marginal improvement over XGBoost

In [39]:
pd.crosstab(y_test, y_predict)

col_0,0,1
CHURN-FLAG,,
0,1191,9
1,45,141


**Further scope of analysis:**   
These classification models assume threshold of 0.5 to classify the outcome.  
It is ok to mis classify a satisfied customer as unsatisfied customer, but not vice versa.  
This is called recall in ML jargon.  
To achieve good recall, we need to re assess our model by chaning classification threshold to values other than 0.5.  
It is also noted that as recall improves, the accuracy of the model decreases.  
We need to comeup with right threshold for which accuracy and recall are tolerable.  
The ML trained object, say model.predict() returns the prediction for the given data with 0.5 as threshold.  
We can use model.predict_proba() to know the probability of prediction being class_1 or class_0.  